<a href="https://colab.research.google.com/github/DanielaNik/lunar-landings-app/blob/master/NASA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#start here
#shift+enter in this box will run the code that's here and open a new code-box below
#import ....

#Note: Imports for the modules that we need
import numpy as np
import pandas as pd

#Note: Requests is a simple, yet elegant, HTTP library.
#Resource: https://pypi.org/project/requests/
import requests

#Note: Beautiful Soup is a library that makes it easy to scrape information from web pages.
#Note: In the following resource you can find a way to select any part of a HTML content.
#Resource: https://pypi.org/project/beautifulsoup4/
from bs4 import BeautifulSoup

#Note: IPython is a command shell for interactive computing in Python.
#Note: IPython.display is a module for display tools in IPython.
#Resource: https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html
from IPython.display import HTML


#Note: We can disable warnings for our requests.
import warnings

requests.packages.urllib3.disable_warnings()
warnings.filterwarnings("ignore")

import re
import json

In [2]:
#Web Scarping from NASA

scraping_url = 'https://www.lroc.asu.edu/featured_sites/'

In [3]:
#Response from site
response = requests.get(scraping_url)
response

<Response [200]>

In [4]:
#Parsing with BeautifulSoup

raw_html = response.text
html = BeautifulSoup(raw_html, "html.parser")
type(html)

bs4.BeautifulSoup

In [5]:
#Save in Drive

from google.colab import drive

drive.mount('/content/drive')
path = "/content/drive/MyDrive/OpenData-Project"

Mounted at /content/drive


In [14]:
html.select_one('.modal-header > h4').text

'GRAIL A'

In [23]:
def get_data(url, path):
  response = requests.get(url)
  html = BeautifulSoup(response.text, "html.parser")

  shuttle_items = html.find_all('div', class_='modal-content')
  lon_items = html.select('.lat-lon')

  shuttles = []

  for i in range(0, len(shuttle_items)):
    item = {}

    shuttle_name = shuttle_items[i].select_one('.modal-header > h4').text.replace('\n','')
    item['Name'] = shuttle_name

    coordinates_text = lon_items[i].text.replace('\n','')
    coordinates_parts = coordinates_text.split()
        
    item['Latitude'] = coordinates_parts[1]
    item['Longitude'] = coordinates_parts[3]
    item['Elevation'] = coordinates_parts[5]

    shuttle_country = shuttle_items[i].select_one('.modal-body > p:nth-child(1)').text.replace('\n','')
    item['Country'] = shuttle_country

    shuttle_description= shuttle_items[i].select_one('.modal-body > p:nth-child(2)').text.replace('\n','')
    item['Description'] = shuttle_description

    shuttles.append(item)


  df = pd.DataFrame(shuttles)
  df['Latitude'] = df['Latitude'].str.replace('°', '')
  df['Longitude'] = df['Longitude'].str.replace('°', '')
  url_parts = url.split("/")
  file_name = path + "/results_" + url_parts[3] + ".csv"
  df.to_csv(file_name, index=False)

  return df

In [24]:
nasa_shuttles_url = "https://www.lroc.asu.edu/featured_sites/"
nasa_data = get_data(nasa_shuttles_url,path)

In [25]:

print(nasa_data)

                      Name                                        Coordinates  \
0                  GRAIL A                    Longitude: 333.41°          ...   
1                  GRAIL B                    Longitude: 333.17°          ...   
2                    LADEE                    Longitude: 266.75°          ...   
3   Chang'e 3 Landing Site                    Longitude: 340.49°          ...   
4   Chang'e 4 Landing Site                    Longitude: 177.59°          ...   
5      17 March 2013 Event                    Longitude: 335.67°          ...   
6           New Crater 260                    Longitude: 319.50°          ...   
7           New Crater 278                    Longitude: 125.41°          ...   
8           New Crater 572                    Longitude: 30.54°           ...   
9                Apollo 11                    Longitude: 23.47°           ...   
10               Apollo 12                    Longitude: 336.58°          ...   
11               Apollo 14  